In [24]:
# @title Show job status
#!pip install paramiko

import ipywidgets as widgets
from IPython.display import HTML, display, Javascript
import getpass
import paramiko
import stat
import os
import time as timer
import subprocess
from urllib.parse import parse_qs
import sys
from urllib.parse import urlparse, parse_qs
import requests
%run "Ed2_helpers.ipynb"

hostname =""
username =""
job_id =""
pkey = ""
remote_output_folder = ""
%store -r hostname
%store -r username
%store -r job_id
%store -r pkey
%store -r remote_output_folder

cluster_input = widgets.Text(value=hostname, description="Cluster:")
username_input = widgets.Text(value=username, description="User name:")
user_password = widgets.Password(placeholder='Enter password',
    description='Password:',
    disabled=False
)
user_pkey = widgets.Text(value=pkey, placeholder='Enter path to private key',
    description='Private key:',
    disabled=False
)
job_id_input = widgets.Text(value=job_id, description="Enter job id:")
output_folder_input = widgets.Text(value=remote_output_folder, description="Enter output folder path:")

input_accordion = widgets.Accordion(children=[widgets.VBox([cluster_input, username_input, user_password, user_pkey, job_id_input, output_folder_input])])
input_accordion.set_title(0, 'Enter details to see job status/output')
display(input_accordion)
## Validate user inputs
validate_button = widgets.Button(description="Validate")
validate_button.on_click(on_validate_button_click2)

validate_accordion = widgets.Accordion(children=[widgets.VBox([validate_button])])
validate_accordion.set_title(0, 'Validate')
display(validate_accordion)
display(output)

no stored variable or alias pkey


Accordion(children=(VBox(children=(Text(value='cc-login.campuscluster.illinois.edu', description='Cluster:'), …

Accordion(children=(VBox(children=(Button(description='Validate', style=ButtonStyle()),)),), titles=('Validate…

Output()

In [25]:
showJobStatus()